In [1]:
%config Application.log_level="INFO"

import logging
from models import *
from train_lstm import *
from ax.service.managed_loop import optimize
import pickle

class ParamSearch:

    def __init__(self, x_train, y_train, x_validate, y_validate, epochs):
        self.x_train = x_train
        self.y_train = y_train
        self.x_validate = x_validate
        self.y_validate = y_validate
        self.epochs = epochs

    def _train_eval(self, params: Dict[str, Union[str, bool, float, int, None]]) -> float:
        lstm = AutoRegressiveLinear(60, 12)
        lstm.cuda()
        mod = NamedModule(lstm, f'paramsearch_{"_".join([str(params[k]) for k in params])}')
        train_epochs(mod, self.x_train, self.y_train, x_validate, y_validate, epochs=self.epochs, **params)
        loss = evaluate(lstm, self.x_validate, self.y_validate)
        if not np.isfinite(loss):
            loss = 1e9
        return loss

    def run(self):
        best_parameters, values, experiment, model = optimize(
            minimize=True,
            parameters=[
                {"name": "lr", "type": "range", "bounds": [1e-5, 1e-2], "log_scale": True},
                {"name": "beta1", "type": "range", "bounds": [0.0, 0.99999], "log_scale": False},
                {"name": "beta2", "type": "range", "bounds": [0.0, 0.99999], "log_scale": False},
                {"name": "batch_size", "type": "range", "bounds": [10, 100], "log_scale": False},
            ],
            evaluation_function=self._train_eval,
            objective_name='mse',
            total_trials=64
        )
        print("Found best parameters: %s", best_parameters)
        return best_parameters, values, experiment, model

In [2]:
from src.train_lstm import *
from ax.plot.contour import plot_contour
from ax.utils.notebook.plotting import render, init_notebook_plotting

warmup = 10
x, y = load_dataset('data/simulations/2021-09-24_time_series.npz', warmup=10)
x, y = subsample(500, x, y)
x = x[:61]
y = y[:50]
x_train, x_validate, x_test, y_train, y_validate, y_test = split_dataset_deterministic(x, y, 0.8, 0.2, 0.0)
x_train, x_validate, y_train, y_validate = to_torch(x_train, x_validate, y_train, y_validate)

logging.info("Starting parameter search.")

s = ParamSearch(x_train, y_train, x_validate, y_validate, 2_000)
best_params, values, experiment, model = s.run()

print('=== BEST PARAMS ===')
print(best_params)

print('=== VALUES ===')
print(values)


[INFO 09-25 16:40:18] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lr. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-25 16:40:18] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter beta1. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-25 16:40:18] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter beta2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-25 16:40:18] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter batch_size. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.


Found best parameters: %s {'lr': 0.001595927839543812, 'beta1': 0.5475287299898006, 'beta2': 0.2710715336156402, 'batch_size': 99}
=== BEST PARAMS ===
{'lr': 0.001595927839543812, 'beta1': 0.5475287299898006, 'beta2': 0.2710715336156402, 'batch_size': 99}
=== VALUES ===
({'mse': -2326.323024800655}, {'mse': {'mse': 83801816.83735661}})


In [16]:
from ax.utils.notebook.plotting import render, init_notebook_plotting

render(plot_contour(model=model, param_x='lr', param_y='batch_size', metric_name='mse', lower_is_better=True,
                    slice_values={'beta1': 0.5475287299898006, 'beta2': 0.2710715336156402}))


In [17]:
render(plot_contour(model=model, param_x='beta1', param_y='beta2', metric_name='mse', lower_is_better=True,
                    slice_values={'lr': 0.001595927839543812, 'batch_size': 99}))